In [1]:
from parameters.dataset import BPI2012Parameters
from dataset import BPI2012Dataset
from utils.print import print_block
from utils.bpi2012 import remove_trail_steps, print_model_prediction_result, remove_tags_for_seq
import tensorflow as tf
from model import LSTMPredNextModel, LSTMScenarioCfModel
import numpy as np
from dice import EventLogDiCE

In [2]:
dataset = BPI2012Dataset(BPI2012Parameters()) 


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/A_CompleteOnly 


In [3]:
trace_idx = 52
_, example_activities, _, example_resources, example_amount, _ = dataset.collate_fn([trace_idx])
print_block(dataset.activity_vocab.list_of_index_to_vocab_2d(example_activities),"Activity")
print_block(dataset.resource_vocab.list_of_index_to_vocab_2d(example_resources),"Resource")


====================Activity====================
| [['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE']] 

====================Resource====================
| [['<SOS>', '112', '112', '112', '11201', '11201', '10629', '10629', '10629']] 


In [4]:
## Remove tails
tail_length_to_remove = 3
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = dataset.activity_vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
example_vocab_resources = dataset.resource_vocab.list_of_index_to_vocab_2d(example_idx_resources)[0]
print_block(example_vocab_activities, "Activity (Tail removed)")
print_block(example_vocab_resources, "Resource (Tail removed)")


====================Activity (Tail removed)====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE'] 

====================Resource (Tail removed)====================
| ['<SOS>', '112', '112', '112', '11201', '11201'] 


In [5]:
pred_model = LSTMPredNextModel.load(
    "./SavedModels/%s" % (
    "0.8441_LSTMPredNextModel_A_CompleteOnly_2021-07-06 16:46:36.809124" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.8441_LSTMPredNextModel_A_CompleteOnly_2021-07-06 16:46:36.809124  

| Vocab loaded successfully from: ./SavedModels/0.8441_LSTMPredNextModel_A_CompleteOnly_2021-07-06 16:46:36.809124  

| Model loaded successfully from: ./SavedModels/0.8441_LSTMPredNextModel_A_CompleteOnly_2021-07-06 16:46:36.809124  


In [6]:
_ = pred_model(**pred_model.get_example_input())
pred_model.summary()

Model: "LSTMPredNextModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  416       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  8192      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (LSTM)                multiple                  49408     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33024     
_________________________________________________________________
sequential (Sequential)      (1, 64)             

In [7]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(pred_model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.29) is "A_CANCELLED_COMPLETE" 



<PAD>                         0.000014
<EOS>                         0.129297
<SOS>                         0.000019
A_ACCEPTED_COMPLETE           0.000103
A_ACTIVATED_COMPLETE          0.196369
A_APPROVED_COMPLETE           0.170411
A_CANCELLED_COMPLETE          0.294768
A_DECLINED_COMPLETE           0.068447
A_FINALIZED_COMPLETE          0.000009
A_PARTLYSUBMITTED_COMPLETE    0.000004
A_PREACCEPTED_COMPLETE        0.000042
A_REGISTERED_COMPLETE         0.140504
A_SUBMITTED_COMPLETE          0.000013
Name: 0, dtype: float64


In [8]:
predicted_df

,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,A_PREACCEPTED_COMPLETE,A_REGISTERED_COMPLETE,A_SUBMITTED_COMPLETE
0,0.000014,0.129297,0.000019,0.000103,0.196369,0.170411,0.294768,0.068447,0.000009,0.000004,0.000042,0.140504,0.000013


In [9]:
no_need_tags = ['<EOS>', '<SOS>', '<PAD>']

possible_activities = [ a for a in list(pred_model.activity_vocab.vocabs) if  not a in no_need_tags]
possible_resources =  [ r for r in list(pred_model.resource_vocab.vocabs) if  not r in no_need_tags]

example_vocab_activities_no_tag = remove_tags_for_seq(example_vocab_activities, no_need_tags)

example_vocab_resources_no_tag = remove_tags_for_seq(example_vocab_resources, no_need_tags)

# transform to possible dim
example_idx_activities_no_tag = [ possible_activities.index(v)   for v in example_vocab_activities_no_tag ]

example_idx_resources_no_tag = [
possible_resources.index(v)   for v in example_vocab_resources_no_tag    
]

example_trace_len_no_tag = len(example_vocab_activities_no_tag)
#### Determine feature names for DiCE ####
activity_feature_names = np.array(["activity_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
resource_feature_names = np.array(["resource_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
possbile_amount = [min(dataset.df["amount"]), max(dataset.df["amount"])]

print_block(example_vocab_activities_no_tag ,title="Example activities without tags", num_marks = 30)
print_block(example_vocab_resources_no_tag ,title="Example resources without tags", num_marks = 30)
print_block(example_amount, title="=================Amount=================", num_marks=16)


==============================Example activities without tags==============================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE'] 

==============================Example resources without tags==============================
| ['112', '112', '112', '11201', '11201'] 

=================================Amount=================================
| [15500.0] 


In [10]:
scenario_model = LSTMScenarioCfModel.load(
    "./SavedModels/%s" % (
        "0.9638_LSTMScenarioCfModel_A_CompleteOnly_5_times_2021-07-06 17:13:29.728576" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.9638_LSTMScenarioCfModel_A_CompleteOnly_5_times_2021-07-06 17:13:29.728576  

| Vocab loaded successfully from: ./SavedModels/0.9638_LSTMScenarioCfModel_A_CompleteOnly_5_times_2021-07-06 17:13:29.728576  

| Model loaded successfully from: ./SavedModels/0.9638_LSTMScenarioCfModel_A_CompleteOnly_5_times_2021-07-06 17:13:29.728576  


In [11]:
_ = scenario_model(**scenario_model.get_example_input())
scenario_model.summary()

Model: "LSTMScenarioCfModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  416       
_________________________________________________________________
embedding_3 (Embedding)      multiple                  8192      
_________________________________________________________________
lstm_4 (LSTM)                multiple                  24832     
_________________________________________________________________
lstm_5 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_6 (LSTM)                multiple                  49408     
_________________________________________________________________
lstm_7 (LSTM)                multiple                  33024     
_________________________________________________________________
sequential_2 (Sequential)    (1, 1, 1)         

In [12]:
dice = EventLogDiCE(
    dataset.activity_vocab,
    dataset.resource_vocab,
    possbile_amount,
    possible_activities,
    possible_resources, 
    pred_model,
    scenario_model
    )

In [32]:
cf_out = dice.run_pls(
    ## Input
    example_amount_input.numpy(),
    example_idx_activities_no_tag,
    example_idx_resources_no_tag,
    desired_vocab = "A_FINALIZED_COMPLETE",
    
    ## Weight
    class_loss_weight = 1,
    scenario_weight = 2e+2,
    distance_loss_weight = 1e-8,
    cat_loss_weight = 1e-3,
    
    ## Training parameters
    scenario_threshold = 0.3,
    scenario_ask_to_continue=True,
    max_iter=1000,
    lr=0.5,

    ## Updating fields
    updating_amount_cf=False,
    # updating_resource_cf=False,
    updating_activity_cf=False,
    
    ## Options
    use_valid_cf_only=False,
    use_sampling=True,
    class_using_hinge_loss=False,
    scenario_using_hinge_loss=False,
    use_clipping=False, 
)

2.26] | Distance [8.96] 

====================Step 40 CF predicted====================
| Invalid: A_CANCELLED_COMPLETE (6) | Valid: A_CANCELLED_COMPLETE (6) 

====================Step 60 Loss====================
| Total [73.84] | Scenario [0.28] | Class [17.91] | Category [56.22] | Distance [9.20] 

====================Step 60 CF predicted====================
| Invalid: A_CANCELLED_COMPLETE (6) | Valid: A_CANCELLED_COMPLETE (6) 

====================Step 80 Loss====================
| Total [73.84] | Scenario [0.28] | Class [17.91] | Category [60.15] | Distance [9.44] 

====================Step 80 CF predicted====================
| Invalid: A_CANCELLED_COMPLETE (6) | Valid: A_CANCELLED_COMPLETE (6) 

====================Step 100 Loss====================
| Total [73.85] | Scenario [0.28] | Class [17.91] | Category [64.05] | Distance [9.69] 

====================Step 100 CF predicted====================
| Invalid: A_CANCELLED_COMPLETE (6) | Valid: A_CANCELLED_COMPLETE (6) 

==============